#### DEFINE DATA LOADER FUNCTIONS

In [1]:
#### DEFINE DATA LOADER FUNCTIONS ####

from torch.utils.data import Dataset

import numpy as np
from scipy.stats import ortho_group

class DataGeneratorPPCA(Dataset):

    def __init__(self, dims, hdims, min_sv=0.11, max_sv=5.0, sigma_sq=0.1, deterministic=True, total=10000):
        self.dims = dims
        self.hdims = hdims

        self.eigs = min_sv + (max_sv - min_sv) * np.linspace(0, 1, hdims)
        self.eigvectors = ortho_group.rvs(dims)[:, :hdims]
        self.w = np.matmul(self.eigvectors, np.diag(np.sqrt(self.eigs - sigma_sq)))

        self.sigma_sq = sigma_sq
        self.sigma = np.sqrt(sigma_sq)

        self.total = total
        self.deterministic = deterministic
        if self.deterministic:
            self.z_sample = np.random.normal(size=(total, self.hdims))
            self.x_sample = np.random.normal(np.matmul(self.z_sample, self.w.T), self.sigma).astype(np.float32)

    def __getitem__(self, i):
        if self.deterministic:
            return self.x_sample[i]
        else:
            z_sample = np.random.normal(size=self.hdims)
            return np.random.normal(self.w.dot(z_sample), self.sigma).astype(np.float32)

    def __len__(self):
        # Return a large number for an epoch
        return self.total


class DataGeneratorPCA(Dataset):
    def __init__(self, dims, hdims, min_sv=0.11, max_sv=5.0, total=10000, sv_list=None,
                 load_data=None):
        self.dims = dims
        self.hdims = hdims

        if load_data is None:
            if isinstance(sv_list, list):
                assert len(sv_list) == dims
                self.full_eigs = np.array(sorted(sv_list, reverse=True))
            else:
                self.full_eigs = min_sv + (max_sv - min_sv) * np.linspace(1, 0, dims)
            self.eigs = self.full_eigs[:hdims]

            self.full_svs = np.sqrt(self.full_eigs)

            self.full_eigvectors = ortho_group.rvs(dims)
            self.eigvectors = self.full_eigvectors[:, :hdims]

            self.total = total

            self.full_z_sample = np.random.normal(size=(total, self.dims))
            self.x_sample = (self.full_eigvectors @ np.diag(self.full_svs) @ self.full_z_sample.T).T.astype(np.float32)

        else:
            self.x_sample = load_data
            u, s, vh = np.linalg.svd(self.x_sample.T, full_matrices=False)
            self.eigs = s[:self.hdims]
            self.eigvectors = u[:, :self.hdims]
            self.total = len(self.x_sample)

    def __getitem__(self, i):
        return self.x_sample[i]

    def __len__(self):
        return self.total

    @property
    def shape(self):
        return self.x_sample.shape


#### DEFINE MODEL CLASSES

In [2]:
#### DEFINE MODEL CLASSES ####

import os
import torch
import torch.nn as nn
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ModelConfig:
    def __init__(self, model_name, model_type, model_class, input_dim, hidden_dim, init_scale, optim_class, lr,
                 extra_model_args={}, extra_optim_args={}):
        self.model_name = model_name
        self.model_type = model_type
        self.model_class = model_class
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.init_scale = init_scale
        self.extra_model_args = extra_model_args

        self.optim_class = optim_class
        self.lr = lr
        self.extra_optim_args = extra_optim_args
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = model_class(input_dim=input_dim, hidden_dim=hidden_dim, init_scale=init_scale, **extra_model_args).to(device)

        self.optimizer = optim_class(self.model.parameters(), lr=lr, **extra_optim_args)

    @property
    def name(self):
        return self.model_name

    @property
    def type(self):
        return self.model_type

    def get_model(self):
        return self.model

    def get_optimizer(self):
        return self.optimizer

class LinearAE(nn.Module):
    def __init__(self,
                 input_dim, hidden_dim, init_scale=0.001,
                 weight_reg_type=None, l2_reg_list=None):
        super(LinearAE, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.encoder = nn.Linear(input_dim, hidden_dim, bias=False)
        self.decoder = nn.Linear(hidden_dim, input_dim, bias=False)

        self.weight_reg_type = weight_reg_type
        self.l2_reg_scalar = None
        self.l2_reg_list = l2_reg_list

        self.encoder.weight.data.normal_(0.0, init_scale)
        self.decoder.weight.data.normal_(0.0, init_scale)

        # configure regularization parameters

        assert self.weight_reg_type is None or isinstance(self.l2_reg_list, list), \
            "l2_reg_list must be a list if weight_reg_type is not None"

        assert self.l2_reg_list is None or len(self.l2_reg_list) == hidden_dim, \
            "Length of l2_reg_list must match latent dimension"

        if weight_reg_type in ("uniform_product", "uniform_sum"):
            self.l2_reg_scalar = l2_reg_list[0] ** 2    # more efficient to use scalar than diag_weights

        elif weight_reg_type == "non_uniform_sum":
            self.reg_weights = torch.tensor(
                np.array(self.l2_reg_list).astype(np.float32)
            )
            self.diag_weights = nn.Parameter(torch.diag(self.reg_weights), requires_grad=False)

    def forward(self, x):
        return self.get_reconstruction_loss(x) + self._get_reg_loss()

    def compute_trace_norm(self):
        """
        Computes the trace norm of the autoencoder, as well as decoder and encoder individually
        :return: trace_norm(W2W1), trace_norm(W1), trace_norm(W2)
        """
        return torch.matmul(self.decoder.weight, self.encoder.weight).norm(p='nuc'), \
               self.encoder.weight.norm(p='nuc'), \
               self.decoder.weight.norm(p='nuc'),

    def get_reconstruction_loss(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)

        recon_loss = torch.sum((x - recon) ** 2) / len(x)
        return recon_loss

    def get_reg_weights_np(self):
        if self.weight_reg_type is None:
            return np.zeros(self.hidden_dim)
        return np.array(self.l2_reg_list)

    def _get_reg_loss(self):
        # Standard L2 regularization, applied to W2W1 (product loss)
        if self.weight_reg_type == 'uniform_product':
            return self.l2_reg_scalar * (torch.norm(torch.matmul(self.decoder.weight, self.encoder.weight)) ** 2)

        # Standard L2 regularization for encoder and decoder separately (sum loss)
        elif self.weight_reg_type == 'uniform_sum':
            # regularize both encoder and decoder
            return self.l2_reg_scalar * (torch.norm(self.encoder.weight) ** 2 + torch.norm(self.decoder.weight) ** 2)

        # non-uniform sum
        elif self.weight_reg_type == 'non_uniform_sum':
            return torch.norm(self.diag_weights @ self.encoder.weight) ** 2 \
                   + torch.norm(self.decoder.weight @ self.diag_weights) ** 2

        # Do not apply regularization
        elif self.weight_reg_type is None:
            return 0.0

        else:
            raise ValueError("weight_reg_type should be one of (uniform_product, uniform_sum, non_uniform_sum, None)")
            

            
            
class LinearAENestedDropout(nn.Module):
    def __init__(self,
                 input_dim, hidden_dim, init_scale=0.001, prior_probs=None, use_expectation=False):
        super(LinearAENestedDropout, self).__init__()

        self.use_expectation = use_expectation

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.encoder = nn.Linear(input_dim, hidden_dim, bias=False)
        self.decoder = nn.Linear(hidden_dim, input_dim, bias=False)

        self.encoder.weight.data.normal_(0.0, init_scale)
        self.decoder.weight.data.normal_(0.0, init_scale)

        if prior_probs is None:
            # use geometric distribution
            # p(b) = rho^b (1 - rho) (b = 0 ... k - 2)
            # p(b = k-1) = 1 - sum(p(b), b < k-1)

            self.geom_p = 0.9
            prior_probs = [self.geom_p ** b * (1 - self.geom_p) for b in range(self.hidden_dim - 1)]
            prior_probs.append(1.0 - sum(prior_probs))

        self.prior_probs = torch.tensor(prior_probs)

        cum_probs = [1. - sum(prior_probs[:i]) for i in range(self.hidden_dim)]
        self.cum_probs = torch.tensor(cum_probs)
        self.diag_expected_mask = nn.Parameter(torch.diag(self.cum_probs), requires_grad=False)
        l_expected_mask = np.zeros((self.hidden_dim, self.hidden_dim))
        for i in range(self.hidden_dim):
            l_expected_mask[i, i] = cum_probs[i]
            l_expected_mask[:i, i] = cum_probs[i]
            l_expected_mask[i, :i] = cum_probs[i]
        self.l_expected_mask = nn.Parameter(torch.from_numpy(l_expected_mask).float(), requires_grad=False)

    def forward(self, x):
        if self.use_expectation:
            tr_xtx = torch.norm(x) ** 2
            w1_x = self.encoder(x).T        # (k, n)
            tr_xt_w2_y = torch.trace(w1_x @ x @ self.decoder.weight @ self.diag_expected_mask)
            w2t_w2_masked = (self.decoder.weight.T @ self.decoder.weight) * self.l_expected_mask
            tr_yt_w2t_w2_y = torch.trace(w1_x @ w1_x.T @ w2t_w2_masked)

            recon_loss = (tr_xtx - 2 * tr_xt_w2_y + tr_yt_w2t_w2_y) / len(x)
        else:
            hidden_units = self.encoder(x)
            hidden_units = self._nested_dropout(hidden_units)
            recon = self.decoder(hidden_units)

            recon_loss = torch.sum((x - recon) ** 2) / len(x)
        return recon_loss

    def _nested_dropout(self, hidden_units):
        prior_inds = torch.multinomial(self.prior_probs, len(hidden_units), replacement=True)
        mask = torch.ones_like(hidden_units)
        for hdim_i in range(1, self.hidden_dim):
            drop_row_inds = (prior_inds < hdim_i).float()     # 1 if row is dropped, 0 if kept
            mask[:, hdim_i] = 1 - drop_row_inds     # 1 if kept, 0 if dropped

        masked_hidden_units = hidden_units * mask
        return masked_hidden_units

    def get_reconstruction_loss(self, x):
        hidden_units = self.encoder(x)
        recon = self.decoder(hidden_units)

        recon_loss = torch.sum((x - recon) ** 2) / len(x)
        return recon_loss
    


#### DEFINE MODEL TRAINING FUNCTION train_models

In [3]:
#### DEFINE MODEL TRAINING FUNCTION train_models ####

import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_models(data_loader, train_itr, metrics_dict, model_configs, eval_metrics_list=None):
    # Initialize model
#     print('new')
#     model = model_config.get_model()
#     optimizer = model_config.get_optimizer()
    
    for train_i in range(train_itr):
        for x in data_loader:
            x_cuda = x.to(device)

            # ---- Optimize ----
            losses = {}

            model = model_config.get_model()
            optimizer = model_config.get_optimizer()

            optimizer.zero_grad()

            loss = model(x_cuda)

            loss.backward()
            
            if model_config.type == 'rotation':
#                 print('Before rotation: ', model.decoder.weight.grad)

                # Rotation Augmented Gradient (RAG) 
                y = model.encoder.weight @ x_cuda.T
                yy_t_norm = y @ y.T / float(len(x))
                yy_t_upper = yy_t_norm - yy_t_norm.tril()
                gamma = 0.5 * (yy_t_upper - yy_t_upper.T)
#                 print('Gamma: ', gamma)
                model.encoder.weight.grad -= gamma @ model.encoder.weight
                model.decoder.weight.grad -= model.decoder.weight @ gamma.T

#                 print('After rotation: ', model.decoder.weight.grad)

            optimizer.step()

            losses[model_config.name] = loss.item()

        # ---- Log statistics ----
        if train_i == 0 or (train_i + 1) % 10 == 0:
            print("".join(["Iteration = {}, Losses: ".format(train_i + 1)]
                          + ["{} = {} ".format(key, val) for key, val in losses.items()]))
            
#     model_config.model = model
    
    return model


In [4]:
# x=next(enumerate(loader))[1]
# x_cuda = x.to(device)
# print(model.encoder.weight.grad)
# y = model.encoder.weight @ x_cuda.T
# yy_t_norm = y @ y.T / float(len(x))
# yy_t_upper = yy_t_norm - yy_t_norm.tril()
# gamma = 0.5 * (yy_t_upper - yy_t_upper.T)
# model.encoder.weight.grad -= gamma @ model.encoder.weight
# print(model.encoder.weight.grad)


#### DEFINE EVALUATION METRICS

In [5]:
import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def get_weight_tensor_from_seq(weight_seq):
    if isinstance(weight_seq, nn.Linear):
        return weight_seq.weight.detach()
    elif isinstance(weight_seq, nn.Sequential):
        weight_tensor = None
        for layer in weight_seq:
            if isinstance(layer, nn.Linear):
                layer_weight = layer.weight.detach()
                if weight_tensor is None:
                    weight_tensor = layer_weight
                else:
                    weight_tensor = layer_weight @ weight_tensor
            elif isinstance(layer, nn.BatchNorm1d):
                bn_weight = layer.weight.detach()

                # ignore bias

                if weight_tensor is None:
                    weight_tensor = torch.diag(bn_weight)
                else:
                    weight_tensor = torch.diag(bn_weight) @ weight_tensor
            else:
                raise ValueError("Layer type {} not supported!".format(type(layer)))
        return weight_tensor


def metric_transpose_theorem(model):
    """
    Metric for how close encoder and decoder.T are
    :param model: LinearAE model
    :return: ||W1 - W2^T||_F^2 / hidden_dim
    """
    encoder_weight = get_weight_tensor_from_seq(model.encoder)
    decoder_weight = get_weight_tensor_from_seq(model.decoder)

    transpose_metric = torch.norm(encoder_weight - decoder_weight.T) ** 2
    return transpose_metric.item() / float(model.hidden_dim)


def metric_alignment(model, gt_eigvectors):
    """
    Metric for alignment of decoder columns to ground truth eigenvectors
    :param model: Linear AE model
    :param gt_eigvectors: ground truth eigenvectors (input_dims,hidden_dims)
    :return: sum_i (1 - max_j (cos(eigvector_i, normalized_decoder column_j)))
    """
    decoder_weight = get_weight_tensor_from_seq(model.decoder)
    decoder_np = decoder_weight.detach().cpu().numpy()

    # normalize columns of gt_eigvectors
    norm_gt_eigvectors = gt_eigvectors / np.linalg.norm(gt_eigvectors, axis=0)
    # normalize columns of decoder
    norm_decoder = decoder_np / (np.linalg.norm(decoder_np, axis=0) + 1e-8)

    total_angles = 0.0
    for eig_i in range(gt_eigvectors.shape[1]):
        eigvector = norm_gt_eigvectors[:, eig_i]
        total_angles += 1. - np.max(np.abs(norm_decoder.T @ eigvector)) ** 2

    return total_angles / float(model.hidden_dim)


def metric_subspace(model, gt_eigvectors, gt_eigs):
    decoder_weight = get_weight_tensor_from_seq(model.decoder)
    decoder_np = decoder_weight.detach().cpu().numpy()

    # k - tr(UU^T WW^T), where W is left singular vector matrix of decoder
    u, s, vh = np.linalg.svd(decoder_np, full_matrices=False)
    return 1 - np.trace(gt_eigvectors @ gt_eigvectors.T @ u @ u.T) / float(model.hidden_dim)


def metric_loss(model, data_loader):
    """
    Measures the full batch loss
    :param model: a linear (variational) AE model
    :param data_loader: full batch data loader. Should be different from the training data loader, if in minibatch mode
    """
    loss = None
    for x in data_loader:
        loss = model(x.to(device)).item()
    return loss


def metric_recon_loss(model, data_loader):
    recon_loss = None
    for x in data_loader:
        recon_loss = model.get_reconstruction_loss(x.to(device)).item()
    return recon_loss


### TRAIN A MODEL

#### Get the data

In [6]:
##### GET DATA ####
import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seed=1234
# set random seed
np.random.seed(seed)
torch.manual_seed(seed)

input_dim = 1000
hidden_dim = 400

n_data = 5000
batch_size = n_data

max_sv = float(input_dim) * 0.1
min_sv = 1.0
sigma = 0.5

gt_data = DataGeneratorPCA(input_dim, hidden_dim, min_sv=min_sv, max_sv=max_sv, total=n_data)
data = DataGeneratorPCA(input_dim, hidden_dim, load_data=gt_data.x_sample)

loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False)


#### Define the model

In [7]:
#### Define the model ####

import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### DEFINE MODEL #####
model_dict = dict(
    model_name='rotation',
    model_type='rotation',
    model_class=LinearAE,
    extra_model_args = {"weight_reg_type": None},
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    init_scale=0.0001,
    optim_class=torch.optim.SGD,
    extra_optim_args={'momentum': 0.9, 'nesterov': True},
    lr=0.0001,
#     optim_class=torch.optim.Adam,
#     extra_optim_args={},
#     lr=0.0003,
    train_itr=1000,#50000,
    seed=seed
)

# model config contains the model 
model_config = ModelConfig(
        model_name=model_dict['model_name'],
        model_type=model_dict['model_type'],
        model_class=model_dict['model_class'],
        input_dim=model_dict['input_dim'], 
        hidden_dim=model_dict['hidden_dim'],
        init_scale=model_dict['init_scale'],
        extra_model_args=model_dict['extra_model_args'],
        optim_class=model_dict['optim_class'],
        lr=model_dict['lr'],
        extra_optim_args=model_dict['extra_optim_args']
    )

print(model_dict,'\n')
print(model_config.get_model(),'\n')
print(model_config.get_optimizer())

print('Transpose:', metric_transpose_theorem(model_config.get_model()),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(model_config.get_model(), data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(model_config.get_model(), data.eigvectors, data.eigs),'\n')



{'model_name': 'rotation', 'model_type': 'rotation', 'model_class': <class '__main__.LinearAE'>, 'extra_model_args': {'weight_reg_type': None}, 'input_dim': 1000, 'hidden_dim': 400, 'init_scale': 0.0001, 'optim_class': <class 'torch.optim.sgd.SGD'>, 'extra_optim_args': {'momentum': 0.9, 'nesterov': True}, 'lr': 0.0001, 'train_itr': 1000, 'seed': 1234} 

LinearAE(
  (encoder): Linear(in_features=1000, out_features=400, bias=False)
  (decoder): Linear(in_features=400, out_features=1000, bias=False)
) 

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.0001
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0
)
Transpose: 1.9994410686194896e-05 

Distance to axis-aligned solution: 0.9899812176786509 

Distance to optimal subspace): 0.6012058744201634 



In [8]:
modelIn = model_config.get_model()
print('Reconstrution Loss:', metric_recon_loss(modelIn, loader),'\n') # full batch loss
print('Loss:', metric_loss(modelIn, loader),'\n')
print('Transpose:', metric_transpose_theorem(modelIn),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(modelIn, data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(modelIn, data.eigvectors, data.eigs),'\n')

trained_model = train_models(data_loader=loader, train_itr=100, metrics_dict=None, model_configs=model_config)

modelIn = model_config.get_model()
print('Reconstrution Loss:', metric_recon_loss(modelIn, loader),'\n') # full batch loss
print('Loss:', metric_loss(modelIn, loader),'\n')
print('Transpose:', metric_transpose_theorem(modelIn),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(modelIn, data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(modelIn, data.eigvectors, data.eigs),'\n')



Reconstrution Loss: 50555.9375 

Loss: 50555.9375 

Transpose: 1.9994410686194896e-05 

Distance to axis-aligned solution: 0.9899812176786509 

Distance to optimal subspace): 0.6012058744201634 

Gamma:  tensor([[ 0.0000e+00,  7.7891e-06, -8.5934e-06,  ..., -6.3761e-06,
         -6.1098e-06, -1.5279e-05],
        [-7.7891e-06,  0.0000e+00,  4.4910e-06,  ...,  7.6507e-06,
          2.2580e-05,  4.7529e-06],
        [ 8.5934e-06, -4.4910e-06,  0.0000e+00,  ..., -8.2844e-06,
         -1.0614e-05, -7.0378e-06],
        ...,
        [ 6.3761e-06, -7.6507e-06,  8.2844e-06,  ...,  0.0000e+00,
         -2.1074e-06,  1.2068e-06],
        [ 6.1098e-06, -2.2580e-05,  1.0614e-05,  ...,  2.1074e-06,
          0.0000e+00, -1.4485e-05],
        [ 1.5279e-05, -4.7529e-06,  7.0378e-06,  ..., -1.2068e-06,
          1.4485e-05,  0.0000e+00]], grad_fn=<MulBackward0>)
Iteration = 1, Losses: rotation = 50555.9375 
Gamma:  tensor([[ 0.0000e+00,  7.9253e-06, -8.2184e-06,  ..., -7.0982e-06,
         -5.8463e-0

Gamma:  tensor([[ 0.0000e+00, -8.4320e-06, -2.5722e-05,  ..., -8.1189e-06,
          1.3046e-05, -4.5721e-05],
        [ 8.4320e-06,  0.0000e+00, -2.7197e-05,  ..., -9.3232e-05,
          3.5901e-05,  2.5303e-05],
        [ 2.5722e-05,  2.7197e-05,  0.0000e+00,  ..., -1.1172e-04,
          4.5427e-05, -7.3799e-05],
        ...,
        [ 8.1189e-06,  9.3232e-05,  1.1172e-04,  ...,  0.0000e+00,
         -5.6206e-06, -5.1651e-05],
        [-1.3046e-05, -3.5901e-05, -4.5427e-05,  ...,  5.6206e-06,
          0.0000e+00, -1.1296e-04],
        [ 4.5721e-05, -2.5303e-05,  7.3799e-05,  ...,  5.1651e-05,
          1.1296e-04,  0.0000e+00]], grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000e+00, -1.5545e-05, -3.2187e-05,  ..., -6.6926e-06,
          1.5123e-05, -5.5364e-05],
        [ 1.5545e-05,  0.0000e+00, -3.6151e-05,  ..., -1.2141e-04,
          3.9389e-05,  3.2808e-05],
        [ 3.2187e-05,  3.6151e-05,  0.0000e+00,  ..., -1.4372e-04,
          5.9510e-05, -9.0828e-05],
        ...,
      

Gamma:  tensor([[ 0.0000e+00, -1.9219e-03, -9.0752e-04,  ...,  2.5944e-04,
         -3.6319e-04, -1.6311e-03],
        [ 1.9219e-03,  0.0000e+00, -1.0049e-03,  ..., -3.6253e-03,
         -1.7898e-04,  1.8666e-03],
        [ 9.0752e-04,  1.0049e-03,  0.0000e+00,  ..., -5.3413e-03,
          1.5301e-03, -2.7186e-03],
        ...,
        [-2.5944e-04,  3.6253e-03,  5.3413e-03,  ...,  0.0000e+00,
          5.6667e-05, -3.0591e-03],
        [ 3.6319e-04,  1.7898e-04, -1.5301e-03,  ..., -5.6667e-05,
          0.0000e+00, -4.9320e-03],
        [ 1.6311e-03, -1.8666e-03,  2.7186e-03,  ...,  3.0591e-03,
          4.9320e-03,  0.0000e+00]], grad_fn=<MulBackward0>)
Iteration = 30, Losses: rotation = 50504.01953125 
Gamma:  tensor([[ 0.0000, -0.0026, -0.0012,  ...,  0.0004, -0.0005, -0.0021],
        [ 0.0026,  0.0000, -0.0013,  ..., -0.0046, -0.0003,  0.0025],
        [ 0.0012,  0.0013,  0.0000,  ..., -0.0069,  0.0019, -0.0035],
        ...,
        [-0.0004,  0.0046,  0.0069,  ...,  0.0000,  0.

Gamma:  tensor([[ 0.0000, -0.3452, -0.1128,  ...,  0.1100, -0.0444, -0.2136],
        [ 0.3452,  0.0000, -0.0445,  ..., -0.3283, -0.1489,  0.2678],
        [ 0.1128,  0.0445,  0.0000,  ..., -0.6658,  0.1089, -0.3268],
        ...,
        [-0.1100,  0.3283,  0.6658,  ...,  0.0000,  0.0657, -0.3414],
        [ 0.0444,  0.1489, -0.1089,  ..., -0.0657,  0.0000, -0.5493],
        [ 0.2136, -0.2678,  0.3268,  ...,  0.3414,  0.5493,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000, -0.4285, -0.1407,  ...,  0.1345, -0.0535, -0.2517],
        [ 0.4285,  0.0000, -0.0572,  ..., -0.3970, -0.1971,  0.3236],
        [ 0.1407,  0.0572,  0.0000,  ..., -0.8252,  0.1382, -0.4112],
        ...,
        [-0.1345,  0.3970,  0.8252,  ...,  0.0000,  0.0892, -0.4072],
        [ 0.0535,  0.1971, -0.1382,  ..., -0.0892,  0.0000, -0.6699],
        [ 0.2517, -0.3236,  0.4112,  ...,  0.4072,  0.6699,  0.0000]],
       grad_fn=<MulBackward0>)
Iteration = 50, Losses: rotation = 45504.73828125 
Gam

Gamma:  tensor([[ 0.0000, -1.6949, -0.7395,  ..., -0.4698,  0.7689,  1.3051],
        [ 1.6949,  0.0000, -1.0907,  ..., -0.5486, -0.7683, -0.1145],
        [ 0.7395,  1.0907,  0.0000,  ..., -1.4974,  1.8430, -0.1016],
        ...,
        [ 0.4698,  0.5486,  1.4974,  ...,  0.0000,  0.6682, -0.4160],
        [-0.7689,  0.7683, -1.8430,  ..., -0.6682,  0.0000, -1.1063],
        [-1.3051,  0.1145,  0.1016,  ...,  0.4160,  1.1063,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000, -1.6642, -0.7319,  ..., -0.3729,  0.9490,  1.2150],
        [ 1.6642,  0.0000, -0.9933,  ..., -0.5634, -0.5153, -0.1059],
        [ 0.7319,  0.9933,  0.0000,  ..., -1.3617,  1.7611, -0.0575],
        ...,
        [ 0.3729,  0.5634,  1.3617,  ...,  0.0000,  0.6377, -0.3312],
        [-0.9490,  0.5153, -1.7611,  ..., -0.6377,  0.0000, -1.0407],
        [-1.2150,  0.1059,  0.0575,  ...,  0.3312,  1.0407,  0.0000]],
       grad_fn=<MulBackward0>)
Iteration = 70, Losses: rotation = 28950.736328125 
Ga

Gamma:  tensor([[ 0.0000, -1.5686, -0.5154,  ..., -0.4386,  0.0191, -0.0072],
        [ 1.5686,  0.0000,  0.0982,  ..., -0.2656,  0.3665,  0.1365],
        [ 0.5154, -0.0982,  0.0000,  ..., -0.8401,  0.9262, -0.5503],
        ...,
        [ 0.4386,  0.2656,  0.8401,  ...,  0.0000,  0.4961,  0.1406],
        [-0.0191, -0.3665, -0.9262,  ..., -0.4961,  0.0000, -0.8702],
        [ 0.0072, -0.1365,  0.5503,  ..., -0.1406,  0.8702,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000, -1.5243, -0.4974,  ..., -0.5007, -0.0507,  0.0448],
        [ 1.5243,  0.0000,  0.1008,  ..., -0.2387,  0.2917,  0.1413],
        [ 0.4974, -0.1008,  0.0000,  ..., -0.8109,  0.8693, -0.5488],
        ...,
        [ 0.5007,  0.2387,  0.8109,  ...,  0.0000,  0.5975,  0.1439],
        [ 0.0507, -0.2917, -0.8693,  ..., -0.5975,  0.0000, -0.8747],
        [-0.0448, -0.1413,  0.5488,  ..., -0.1439,  0.8747,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000, -1.4776, -0.4766,  ..., -0.5

In [9]:
##### GET DATA ####
import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seed=1234
# set random seed
np.random.seed(seed)
torch.manual_seed(seed)

input_dim = 1000
hidden_dim = 50

n_data = 5000
batch_size = n_data

max_sv = float(input_dim) * 0.1
min_sv = 1.0
sigma = 0.5

gt_data = DataGeneratorPCA(input_dim, hidden_dim, min_sv=min_sv, max_sv=max_sv, total=n_data)
data = DataGeneratorPCA(input_dim, hidden_dim, load_data=gt_data.x_sample)

loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False)

#### Define the model ####

import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### DEFINE MODEL #####
model_dict = dict(
    model_name='rotation',
    model_type='rotation',
    model_class=LinearAE,
    extra_model_args = {"weight_reg_type": None},
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    init_scale=0.0001,
    optim_class=torch.optim.SGD,
    extra_optim_args={'momentum': 0.9, 'nesterov': True},
    lr=0.0001,
#     optim_class=torch.optim.Adam,
#     extra_optim_args={},
#     lr=0.0003,
    train_itr=1000,#50000,
    seed=seed
)

# model config contains the model 
model_config = ModelConfig(
        model_name=model_dict['model_name'],
        model_type=model_dict['model_type'],
        model_class=model_dict['model_class'],
        input_dim=model_dict['input_dim'], 
        hidden_dim=model_dict['hidden_dim'],
        init_scale=model_dict['init_scale'],
        extra_model_args=model_dict['extra_model_args'],
        optim_class=model_dict['optim_class'],
        lr=model_dict['lr'],
        extra_optim_args=model_dict['extra_optim_args']
    )

print(model_dict,'\n')
print(model_config.get_model(),'\n')
print(model_config.get_optimizer())

print('Transpose:', metric_transpose_theorem(model_config.get_model()),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(model_config.get_model(), data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(model_config.get_model(), data.eigvectors, data.eigs),'\n')



{'model_name': 'rotation', 'model_type': 'rotation', 'model_class': <class '__main__.LinearAE'>, 'extra_model_args': {'weight_reg_type': None}, 'input_dim': 1000, 'hidden_dim': 50, 'init_scale': 0.0001, 'optim_class': <class 'torch.optim.sgd.SGD'>, 'extra_optim_args': {'momentum': 0.9, 'nesterov': True}, 'lr': 0.0001, 'train_itr': 1000, 'seed': 1234} 

LinearAE(
  (encoder): Linear(in_features=1000, out_features=50, bias=False)
  (decoder): Linear(in_features=50, out_features=1000, bias=False)
) 

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.0001
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0
)
Transpose: 1.9993053283542393e-05 

Distance to axis-aligned solution: 0.993799557974467 

Distance to optimal subspace): 0.9513405523538548 



In [10]:
modelIn = model_config.get_model()
print('Reconstrution Loss:', metric_recon_loss(modelIn, loader),'\n') # full batch loss
print('Loss:', metric_loss(modelIn, loader),'\n')
print('Transpose:', metric_transpose_theorem(modelIn),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(modelIn, data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(modelIn, data.eigvectors, data.eigs),'\n')

trained_model = train_models(data_loader=loader, train_itr=1000, metrics_dict=None, model_configs=model_config)

modelIn = model_config.get_model()
print('Reconstrution Loss:', metric_recon_loss(modelIn, loader),'\n') # full batch loss
print('Loss:', metric_loss(modelIn, loader),'\n')
print('Transpose:', metric_transpose_theorem(modelIn),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(modelIn, data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(modelIn, data.eigvectors, data.eigs),'\n')



Reconstrution Loss: 50555.9375 

Loss: 50555.9375 

Transpose: 1.9993053283542393e-05 

Distance to axis-aligned solution: 0.993799557974467 

Distance to optimal subspace): 0.9513405523538548 

Gamma:  tensor([[ 0.0000e+00,  2.4825e-08,  1.1136e-05,  ...,  1.7017e-05,
          6.6775e-06, -5.8729e-06],
        [-2.4825e-08,  0.0000e+00,  1.3265e-05,  ..., -5.7284e-06,
          2.0758e-05, -1.2807e-06],
        [-1.1136e-05, -1.3265e-05,  0.0000e+00,  ...,  2.8009e-06,
         -6.3987e-06, -6.9829e-06],
        ...,
        [-1.7017e-05,  5.7284e-06, -2.8009e-06,  ...,  0.0000e+00,
          1.6798e-06, -5.9501e-06],
        [-6.6775e-06, -2.0758e-05,  6.3987e-06,  ..., -1.6798e-06,
          0.0000e+00,  6.1862e-06],
        [ 5.8729e-06,  1.2807e-06,  6.9829e-06,  ...,  5.9501e-06,
         -6.1862e-06,  0.0000e+00]], grad_fn=<MulBackward0>)
Iteration = 1, Losses: rotation = 50555.9375 
Gamma:  tensor([[ 0.0000e+00,  7.2502e-07,  1.1781e-05,  ...,  1.7055e-05,
          6.4766e-06

Gamma:  tensor([[ 0.0000e+00,  3.4359e-05,  6.6931e-05,  ...,  5.4405e-06,
          1.2831e-05, -5.5701e-05],
        [-3.4359e-05,  0.0000e+00,  5.1028e-05,  ..., -4.6271e-05,
          1.7901e-05,  1.1047e-05],
        [-6.6931e-05, -5.1028e-05,  0.0000e+00,  ...,  5.1058e-05,
         -3.5043e-06,  7.0114e-06],
        ...,
        [-5.4405e-06,  4.6271e-05, -5.1058e-05,  ...,  0.0000e+00,
          4.3875e-05, -4.1429e-05],
        [-1.2831e-05, -1.7901e-05,  3.5043e-06,  ..., -4.3875e-05,
          0.0000e+00, -3.4105e-05],
        [ 5.5701e-05, -1.1047e-05, -7.0114e-06,  ...,  4.1429e-05,
          3.4105e-05,  0.0000e+00]], grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000e+00,  4.3127e-05,  8.2547e-05,  ...,  1.9163e-06,
          1.4856e-05, -6.6221e-05],
        [-4.3127e-05,  0.0000e+00,  6.2539e-05,  ..., -5.9569e-05,
          2.0328e-05,  1.4906e-05],
        [-8.2547e-05, -6.2539e-05,  0.0000e+00,  ...,  6.4217e-05,
         -4.9444e-06,  1.0509e-05],
        ...,
      

Gamma:  tensor([[ 0.0000,  0.0009,  0.0015,  ..., -0.0004, -0.0002, -0.0009],
        [-0.0009,  0.0000,  0.0012,  ..., -0.0016,  0.0005,  0.0007],
        [-0.0015, -0.0012,  0.0000,  ...,  0.0015, -0.0004,  0.0003],
        ...,
        [ 0.0004,  0.0016, -0.0015,  ...,  0.0000,  0.0012, -0.0020],
        [ 0.0002, -0.0005,  0.0004,  ..., -0.0012,  0.0000, -0.0016],
        [ 0.0009, -0.0007, -0.0003,  ...,  0.0020,  0.0016,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000,  0.0011,  0.0019,  ..., -0.0006, -0.0003, -0.0012],
        [-0.0011,  0.0000,  0.0014,  ..., -0.0021,  0.0007,  0.0009],
        [-0.0019, -0.0014,  0.0000,  ...,  0.0019, -0.0006,  0.0004],
        ...,
        [ 0.0006,  0.0021, -0.0019,  ...,  0.0000,  0.0015, -0.0026],
        [ 0.0003, -0.0007,  0.0006,  ..., -0.0015,  0.0000, -0.0021],
        [ 0.0012, -0.0009, -0.0004,  ...,  0.0026,  0.0021,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000,  0.0014,  0.0023,  ..., -0.0

Gamma:  tensor([[ 0.0000,  0.0744,  0.1020,  ..., -0.1044, -0.1140, -0.0671],
        [-0.0744,  0.0000,  0.0965,  ..., -0.1908,  0.1243,  0.1268],
        [-0.1020, -0.0965,  0.0000,  ...,  0.1808, -0.0757,  0.0561],
        ...,
        [ 0.1044,  0.1908, -0.1808,  ...,  0.0000,  0.1338, -0.3157],
        [ 0.1140, -0.1243,  0.0757,  ..., -0.1338,  0.0000, -0.2058],
        [ 0.0671, -0.1268, -0.0561,  ...,  0.3157,  0.2058,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000,  0.0941,  0.1268,  ..., -0.1371, -0.1520, -0.0839],
        [-0.0941,  0.0000,  0.1214,  ..., -0.2450,  0.1642,  0.1649],
        [-0.1268, -0.1214,  0.0000,  ...,  0.2328, -0.0980,  0.0724],
        ...,
        [ 0.1371,  0.2450, -0.2328,  ...,  0.0000,  0.1705, -0.4086],
        [ 0.1520, -0.1642,  0.0980,  ..., -0.1705,  0.0000, -0.2635],
        [ 0.0839, -0.1649, -0.0724,  ...,  0.4086,  0.2635,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000,  0.1187,  0.1575,  ..., -0.1

Gamma:  tensor([[ 0.0000,  0.9428,  3.2963,  ..., -3.0275, -1.6901, -0.4048],
        [-0.9428,  0.0000,  1.7736,  ..., -5.5357,  2.2181,  2.7341],
        [-3.2963, -1.7736,  0.0000,  ...,  4.4944, -2.6574, -0.0698],
        ...,
        [ 3.0275,  5.5357, -4.4944,  ...,  0.0000,  1.2860, -4.0295],
        [ 1.6901, -2.2181,  2.6574,  ..., -1.2860,  0.0000, -4.0672],
        [ 0.4048, -2.7341,  0.0698,  ...,  4.0295,  4.0672,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000,  0.7338,  3.4898,  ..., -3.1017, -1.3185, -0.2377],
        [-0.7338,  0.0000,  1.8108,  ..., -5.6872,  1.8522,  2.6397],
        [-3.4898, -1.8108,  0.0000,  ...,  4.3879, -2.6795, -0.0780],
        ...,
        [ 3.1017,  5.6872, -4.3879,  ...,  0.0000,  0.8471, -3.5859],
        [ 1.3185, -1.8522,  2.6795,  ..., -0.8471,  0.0000, -4.2368],
        [ 0.2377, -2.6397,  0.0780,  ...,  3.5859,  4.2368,  0.0000]],
       grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000,  0.4676,  3.5426,  ..., -3.1

...

Gamma:  tensor([[ 0.0000e+00,  5.1229e-01, -2.3697e-01,  ...,  7.9485e-04,
         -2.9863e-04, -4.8352e-04],
        [-5.1229e-01,  0.0000e+00,  3.7693e-02,  ...,  5.2429e-04,
          3.7817e-04,  3.6431e-04],
        [ 2.3697e-01, -3.7693e-02,  0.0000e+00,  ...,  5.4229e-04,
         -3.8038e-04, -4.1096e-04],
        ...,
        [-7.9485e-04, -5.2429e-04, -5.4229e-04,  ...,  0.0000e+00,
          2.2419e-03,  8.6774e-02],
        [ 2.9863e-04, -3.7817e-04,  3.8038e-04,  ..., -2.2419e-03,
          0.0000e+00, -2.0661e-01],
        [ 4.8352e-04, -3.6431e-04,  4.1096e-04,  ..., -8.6774e-02,
          2.0661e-01,  0.0000e+00]], grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000e+00,  5.1237e-01, -2.3667e-01,  ...,  7.9379e-04,
         -2.9746e-04, -4.8134e-04],
        [-5.1237e-01,  0.0000e+00,  3.7925e-02,  ...,  5.2463e-04,
          3.7630e-04,  3.6299e-04],
        [ 2.3667e-01, -3.7925e-02,  0.0000e+00,  ...,  5.3718e-04,
         -3.7717e-04, -4.0967e-04],
        ...,
      

Gamma:  tensor([[ 0.0000e+00,  5.1333e-01, -2.3308e-01,  ...,  7.7716e-04,
         -2.7861e-04, -4.5079e-04],
        [-5.1333e-01,  0.0000e+00,  4.0641e-02,  ...,  5.1431e-04,
          3.4429e-04,  3.4248e-04],
        [ 2.3308e-01, -4.0641e-02,  0.0000e+00,  ...,  4.9500e-04,
         -3.4645e-04, -3.8640e-04],
        ...,
        [-7.7716e-04, -5.1431e-04, -4.9500e-04,  ...,  0.0000e+00,
          2.7865e-03,  8.8840e-02],
        [ 2.7861e-04, -3.4429e-04,  3.4645e-04,  ..., -2.7865e-03,
          0.0000e+00, -2.0742e-01],
        [ 4.5079e-04, -3.4248e-04,  3.8640e-04,  ..., -8.8840e-02,
          2.0742e-01,  0.0000e+00]], grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000e+00,  5.1340e-01, -2.3278e-01,  ...,  7.7617e-04,
         -2.7627e-04, -4.4940e-04],
        [-5.1340e-01,  0.0000e+00,  4.0862e-02,  ...,  5.1382e-04,
          3.4119e-04,  3.4050e-04],
        [ 2.3278e-01, -4.0862e-02,  0.0000e+00,  ...,  4.9102e-04,
         -3.4413e-04, -3.8508e-04],
        ...,
      

Gamma:  tensor([[ 0.0000e+00,  5.1436e-01, -2.2899e-01,  ...,  7.5754e-04,
         -2.5781e-04, -4.1898e-04],
        [-5.1436e-01,  0.0000e+00,  4.3636e-02,  ...,  5.0354e-04,
          3.1185e-04,  3.2292e-04],
        [ 2.2899e-01, -4.3636e-02,  0.0000e+00,  ...,  4.4844e-04,
         -3.1394e-04, -3.6165e-04],
        ...,
        [-7.5754e-04, -5.0354e-04, -4.4844e-04,  ...,  0.0000e+00,
          3.2611e-03,  9.0944e-02],
        [ 2.5781e-04, -3.1185e-04,  3.1394e-04,  ..., -3.2611e-03,
          0.0000e+00, -2.0823e-01],
        [ 4.1898e-04, -3.2292e-04,  3.6165e-04,  ..., -9.0944e-02,
          2.0823e-01,  0.0000e+00]], grad_fn=<MulBackward0>)
Iteration = 970, Losses: rotation = 44026.48046875 
Gamma:  tensor([[ 0.0000e+00,  5.1443e-01, -2.2870e-01,  ...,  7.5618e-04,
         -2.5620e-04, -4.1677e-04],
        [-5.1443e-01,  0.0000e+00,  4.3842e-02,  ...,  5.0142e-04,
          3.0935e-04,  3.2087e-04],
        [ 2.2870e-01, -4.3842e-02,  0.0000e+00,  ...,  4.4631e-04,
   

Gamma:  tensor([[ 0.0000e+00,  5.1532e-01, -2.2501e-01,  ...,  7.3672e-04,
         -2.3804e-04, -3.8895e-04],
        [-5.1532e-01,  0.0000e+00,  4.6451e-02,  ...,  4.9053e-04,
          2.8498e-04,  3.0396e-04],
        [ 2.2501e-01, -4.6451e-02,  0.0000e+00,  ...,  4.0994e-04,
         -2.8552e-04, -3.3804e-04],
        ...,
        [-7.3672e-04, -4.9053e-04, -4.0994e-04,  ...,  0.0000e+00,
          3.6340e-03,  9.2927e-02],
        [ 2.3804e-04, -2.8498e-04,  2.8552e-04,  ..., -3.6340e-03,
          0.0000e+00, -2.0898e-01],
        [ 3.8895e-04, -3.0396e-04,  3.3804e-04,  ..., -9.2927e-02,
          2.0898e-01,  0.0000e+00]], grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000e+00,  5.1538e-01, -2.2472e-01,  ...,  7.3523e-04,
         -2.3716e-04, -3.8650e-04],
        [-5.1538e-01,  0.0000e+00,  4.6644e-02,  ...,  4.9009e-04,
          2.8157e-04,  3.0217e-04],
        [ 2.2472e-01, -4.6644e-02,  0.0000e+00,  ...,  4.0649e-04,
         -2.8342e-04, -3.3857e-04],
        ...,
      

Gamma:  tensor([[ 0.0000e+00,  5.1618e-01, -2.2112e-01,  ...,  7.1685e-04,
         -2.2109e-04, -3.6249e-04],
        [-5.1618e-01,  0.0000e+00,  4.9089e-02,  ...,  4.7698e-04,
          2.6014e-04,  2.8652e-04],
        [ 2.2112e-01, -4.9089e-02,  0.0000e+00,  ...,  3.7209e-04,
         -2.6082e-04, -3.1763e-04],
        ...,
        [-7.1685e-04, -4.7698e-04, -3.7209e-04,  ...,  0.0000e+00,
          3.9196e-03,  9.4801e-02],
        [ 2.2109e-04, -2.6014e-04,  2.6082e-04,  ..., -3.9196e-03,
          0.0000e+00, -2.0968e-01],
        [ 3.6249e-04, -2.8652e-04,  3.1763e-04,  ..., -9.4801e-02,
          2.0968e-01,  0.0000e+00]], grad_fn=<MulBackward0>)
Gamma:  tensor([[ 0.0000e+00,  5.1624e-01, -2.2085e-01,  ...,  7.1545e-04,
         -2.1963e-04, -3.6140e-04],
        [-5.1624e-01,  0.0000e+00,  4.9271e-02,  ...,  4.7676e-04,
          2.5790e-04,  2.8586e-04],
        [ 2.2085e-01, -4.9271e-02,  0.0000e+00,  ...,  3.7085e-04,
         -2.5837e-04, -3.1606e-04],
        ...,
      

In [11]:
##### GET DATA ####
import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seed=1234
# set random seed
np.random.seed(seed)
torch.manual_seed(seed)

input_dim = 1000
hidden_dim = 20

n_data = 5000
batch_size = n_data

max_sv = float(input_dim) * 0.1
min_sv = 1.0
sigma = 0.5

gt_data = DataGeneratorPCA(input_dim, hidden_dim, min_sv=min_sv, max_sv=max_sv, total=n_data)
data = DataGeneratorPCA(input_dim, hidden_dim, load_data=gt_data.x_sample)

loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False)

#### Define the model ####

import os
import torch
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#### DEFINE MODEL #####
model_dict = dict(
    model_name='nd_expectation',
    model_type='nested_dropout',
    model_class=LinearAENestedDropout,
    extra_model_args = {'use_expectation': True},
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    init_scale=0.0001,
    optim_class=torch.optim.Adam,
    extra_optim_args={},
    lr=0.003,
    train_itr=3000,#50000,
    seed=seed
)

# model config contains the model 
model_config = ModelConfig(
        model_name=model_dict['model_name'],
        model_type=model_dict['model_type'],
        model_class=model_dict['model_class'],
        input_dim=model_dict['input_dim'], 
        hidden_dim=model_dict['hidden_dim'],
        init_scale=model_dict['init_scale'],
        extra_model_args=model_dict['extra_model_args'],
        optim_class=model_dict['optim_class'],
        lr=model_dict['lr'],
        extra_optim_args=model_dict['extra_optim_args']
    )

print(model_dict,'\n')
print(model_config.get_model(),'\n')
print(model_config.get_optimizer())

print('Transpose:', metric_transpose_theorem(model_config.get_model()),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(model_config.get_model(), data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(model_config.get_model(), data.eigvectors, data.eigs),'\n')





{'model_name': 'nd_expectation', 'model_type': 'nested_dropout', 'model_class': <class '__main__.LinearAENestedDropout'>, 'extra_model_args': {'use_expectation': True}, 'input_dim': 1000, 'hidden_dim': 20, 'init_scale': 0.0001, 'optim_class': <class 'torch.optim.adam.Adam'>, 'extra_optim_args': {}, 'lr': 0.003, 'train_itr': 3000, 'seed': 1234} 

LinearAENestedDropout(
  (encoder): Linear(in_features=1000, out_features=20, bias=False)
  (decoder): Linear(in_features=20, out_features=1000, bias=False)
) 

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.003
    maximize: False
    weight_decay: 0
)
Transpose: 2.013526827795431e-05 

Distance to axis-aligned solution: 0.994424752923425 

Distance to optimal subspace): 0.9777534324698326 



In [12]:
modelIn = model_config.get_model()
print('Reconstrution Loss:', metric_recon_loss(modelIn, loader),'\n') # full batch loss
print('Loss:', metric_loss(modelIn, loader),'\n')
print('Transpose:', metric_transpose_theorem(modelIn),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(modelIn, data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(modelIn, data.eigvectors, data.eigs),'\n')

trained_model = train_models(data_loader=loader, train_itr=model_dict['train_itr'], metrics_dict=None, model_configs=model_config)

modelIn = model_config.get_model()
print('Reconstrution Loss:', metric_recon_loss(modelIn, loader),'\n') # full batch loss
print('Loss:', metric_loss(modelIn, loader),'\n')
print('Transpose:', metric_transpose_theorem(modelIn),'\n') # how close encoder and decoder.T are 
print('Distance to axis-aligned solution:', metric_alignment(modelIn, data.eigvectors),'\n') # alignment of decoder columns to ground truth eigenvectors
print('Distance to optimal subspace):', metric_subspace(modelIn, data.eigvectors, data.eigs),'\n')



Reconstrution Loss: 50555.9375 

Loss: 50544.8828125 

Transpose: 2.013526827795431e-05 

Distance to axis-aligned solution: 0.994424752923425 

Distance to optimal subspace): 0.9777534324698326 

Iteration = 1, Losses: nd_expectation = 50544.8828125 
Iteration = 10, Losses: nd_expectation = 50005.375 
Iteration = 20, Losses: nd_expectation = 49659.06640625 
Iteration = 30, Losses: nd_expectation = 49504.6328125 
Iteration = 40, Losses: nd_expectation = 49428.1875 
Iteration = 50, Losses: nd_expectation = 49391.37109375 
Iteration = 60, Losses: nd_expectation = 49371.125 
Iteration = 70, Losses: nd_expectation = 49358.13671875 
Iteration = 80, Losses: nd_expectation = 49350.11328125 
Iteration = 90, Losses: nd_expectation = 49344.70703125 
Iteration = 100, Losses: nd_expectation = 49340.89453125 
Iteration = 110, Losses: nd_expectation = 49338.078125 
Iteration = 120, Losses: nd_expectation = 49335.91796875 
Iteration = 130, Losses: nd_expectation = 49334.1875 
Iteration = 140, Losses:

Iteration = 1400, Losses: nd_expectation = 49319.25390625 
Iteration = 1410, Losses: nd_expectation = 49319.24609375 
Iteration = 1420, Losses: nd_expectation = 49319.24609375 
Iteration = 1430, Losses: nd_expectation = 49319.23828125 
Iteration = 1440, Losses: nd_expectation = 49319.234375 
Iteration = 1450, Losses: nd_expectation = 49319.23046875 
Iteration = 1460, Losses: nd_expectation = 49319.22265625 
Iteration = 1470, Losses: nd_expectation = 49319.21875 
Iteration = 1480, Losses: nd_expectation = 49319.21484375 
Iteration = 1490, Losses: nd_expectation = 49319.2109375 
Iteration = 1500, Losses: nd_expectation = 49319.2109375 
Iteration = 1510, Losses: nd_expectation = 49319.203125 
Iteration = 1520, Losses: nd_expectation = 49319.19921875 
Iteration = 1530, Losses: nd_expectation = 49319.1953125 
Iteration = 1540, Losses: nd_expectation = 49319.19140625 
Iteration = 1550, Losses: nd_expectation = 49319.1875 
Iteration = 1560, Losses: nd_expectation = 49319.1875 
Iteration = 157

Iteration = 2820, Losses: nd_expectation = 49319.02734375 
Iteration = 2830, Losses: nd_expectation = 49319.02734375 
Iteration = 2840, Losses: nd_expectation = 49319.02734375 
Iteration = 2850, Losses: nd_expectation = 49319.02734375 
Iteration = 2860, Losses: nd_expectation = 49319.0234375 
Iteration = 2870, Losses: nd_expectation = 49319.0234375 
Iteration = 2880, Losses: nd_expectation = 49319.02734375 
Iteration = 2890, Losses: nd_expectation = 49319.0234375 
Iteration = 2900, Losses: nd_expectation = 49319.02734375 
Iteration = 2910, Losses: nd_expectation = 49319.0234375 
Iteration = 2920, Losses: nd_expectation = 49319.0234375 
Iteration = 2930, Losses: nd_expectation = 49319.01953125 
Iteration = 2940, Losses: nd_expectation = 49319.0234375 
Iteration = 2950, Losses: nd_expectation = 49319.01953125 
Iteration = 2960, Losses: nd_expectation = 49319.0234375 
Iteration = 2970, Losses: nd_expectation = 49319.0234375 
Iteration = 2980, Losses: nd_expectation = 49319.0234375 
Iterat